# Runtime Test

Before runing this test, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
#  # dynamically grow the memory used on the GPU
# from tensorflow.compat.v1.keras.backend import set_session
# import tensorflow as tf
# #config = tf.compat.v1.ConfigProto()
# #config.gpu_options.allow_growth = True 
# #sess = tf.compat.v1.Session(config=config)
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# set_session(sess)


In [3]:
# Import all necessary libraries.
import numpy as np
import sys
import glob
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
from turbojpeg import TurboJPEG
jpeg = TurboJPEG()

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [4]:
!pip3 install -U git+git://github.com/lilohuang/PyTurboJPEG.git

  Cloning git://github.com/lilohuang/PyTurboJPEG.git to /tmp/pip-req-build-_jsuj57s
  Created wheel for PyTurboJPEG: filename=PyTurboJPEG-1.4.1-py3-none-any.whl size=7846 sha256=3389a4344b99087310d1d1a7887efd601bc21184de9edb0f1193c9d90728638f
  Stored in directory: /tmp/pip-ephem-wheel-cache-wfucytjm/wheels/0b/47/e5/13fc5b97f66f9e83aa0c98f39496df95b023ec2c847ef49c0e
Successfully built PyTurboJPEG
  Attempting uninstall: PyTurboJPEG
    Found existing installation: PyTurboJPEG 1.4.1
    Uninstalling PyTurboJPEG-1.4.1:
      Successfully uninstalled PyTurboJPEG-1.4.1


In [5]:
from NomeroffNet.YoloV5Detector import Detector

detector = Detector()
detector.load()

Fusing layers... 


In [6]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft, convertCvZonesRGBtoBGR, getCvZoneRGB, reshapePoints

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

Loading weights from checkpoint (/var/www/nomeroff-net20/NomeroffNet/Base/mcm/./models/NpPointsCraft/mtl/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/var/www/nomeroff-net20/NomeroffNet/Base/mcm/./models/NpPointsCraft/refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [7]:
from NomeroffNet.OptionsDetector import OptionsDetector
from NomeroffNet.TextDetector import TextDetector

# Import license plate recognition tools.
#from NomeroffNet.RectDetector import  RectDetector
#rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    },
    "su": {
        "for_regions": ["su"],
        "model_path": "latest"
    }
})

In [8]:
async def test(dirName, fname, min_bbox_acc = 0.5):
    start_time = time.time()
    img_path = os.path.join(dirName, fname)
    with open(img_path, 'rb') as in_file:
        img = jpeg.decode(in_file.read())
    #img = cv2.imread(img_path)
    image_load_time = time.time() - start_time

    start_time = time.time()
    targetBoxes = detector.detect_bbox(img)
    detect_bbox_time = time.time() - start_time
    
    start_time = time.time()
    all_points = npPointsCraft.detect(img, targetBoxes)
    all_points = [ps for ps in all_points if len(ps)]
    craft_time = time.time() - start_time
    
    start_time = time.time()
    
    zones = convertCvZonesRGBtoBGR([getCvZoneRGB(img, reshapePoints(rect,1)) for rect in all_points])
    
    perspective_align_time = time.time() - start_time
    
    start_time = time.time()
    regionIds, stateIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    classification_time = time.time() - start_time
    
    start_time = time.time()
    textArr = textDetector.predict(zones, regionNames, countLines)
    ocr_time = time.time() - start_time
    return (image_load_time, detect_bbox_time, craft_time, perspective_align_time, classification_time, ocr_time)

In [14]:
N = 10

i = 0
j = 0

image_load_time_all        = 0
detect_bbox_time_all       = 0
craft_time_all             = 0
perspective_align_time_all = 0
classification_time_all    = 0
ocr_time_all               = 0

start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            image_load_time, \
                detect_bbox_time, \
                craft_time, \
                perspective_align_time, \
                classification_time, \
                ocr_time = await test(dirName, fname)
            image_load_time_all        += image_load_time
            detect_bbox_time_all       += detect_bbox_time
            craft_time_all             += craft_time
            perspective_align_time_all += perspective_align_time
            classification_time_all    += classification_time
            ocr_time_all               += ocr_time
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0
pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7
pass 8
pass 9


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + NVIDIA RTX 3090

In [15]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")
print()
print(f"image_load_time_all {image_load_time_all}; {image_load_time_all/j} per one photo")
print(f"detect_bbox_time_all {detect_bbox_time_all}; {detect_bbox_time_all/j} per one photo")
print(f"craft_time_all {craft_time_all}; {craft_time_all/j} per one photo")
print(f"perspective_align_time_all {perspective_align_time_all}; {perspective_align_time_all/j} per one photo")
print(f"classification_time_all {classification_time_all}; {classification_time_all/j} per one photo")
print(f"ocr_time_all {ocr_time_all}; {ocr_time_all/j} per one photo")

Processed 260 photos
Time 23.244853019714355
One photo process 0.08940328084505521 seconds

image_load_time_all 5.80341100692749; 0.022320811565105732 per one photo
detect_bbox_time_all 5.036606788635254; 0.019371564571674053 per one photo
craft_time_all 6.945644855499268; 0.026714018674997182 per one photo
perspective_align_time_all 0.8101990222930908; 0.003116150085742657 per one photo
classification_time_all 1.0684123039245605; 0.00410927809201754 per one photo
ocr_time_all 3.566481590270996; 0.013717236885657678 per one photo


In [11]:
# keras classification                             time 0.0423
# keras classification with bach size              time 0.0347
# keras classification model()                     time 0.0120
# keras classification model() tr=false            time 0.0120
# keras classification model() tr=false, one batch time 0.0100

In [12]:
# keras ocr                                        time 0.0562
# keras ocr with bach size                         time 0.0537
# keras classification model()                     time 0.0307
# keras classification model() tr=false            time 0.0295
# keras classification model() tr=false, one batch time 0.0284

In [13]:
# cv2 image load time 0.057034785930926986
# image load time 0.030525074555323675

### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + Quadro RTX 8000